# Sequential Simulations with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies
Since the `OperatiotnsProblem` is the fundamental building block of a sequential
simulation in PowerSimulations, we will build on the [OperationsProblem example](../../notebook/3_PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(
    joinpath(pkgpath, "test", "3_PowerSimulations_examples", "01_operations_problems.jl"),
)

[ Info: Loaded time series from storage file existing=modified_RTS_GMLC_DA_sys_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_IqRLtA
┌ Warning: Rate 500.0 MW for C31-2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/4kGrw/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 175.0 MW for B8 is larger than the max expected in the range of (min = 47.0, max = 52.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/4kGrw/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for B26 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/4kGrw/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for A32-2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/4kGrw/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for C

BuildStatus.BUILT = 0

### 5-Minute system
We had already created a `sys::System` from hourly RTS data in the OperationsProblem example.
The RTS data also includes 5-minute resolution time series data. So, we can create another
`System`:

In [2]:
sys_RT = build_system(PSITestSystems, "modified_RTS_GMLC_RT_sys")

[ Info: Loaded time series from storage file existing=modified_RTS_GMLC_RT_sys_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_xQx25B
┌ Warning: Rate 500.0 MW for C31-2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/4kGrw/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 175.0 MW for B8 is larger than the max expected in the range of (min = 47.0, max = 52.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/4kGrw/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for B26 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/4kGrw/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for A32-2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/4kGrw/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 500.0 MW for C

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,109
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,73
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,HVDCLine,DCBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
7,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,19
8,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,105
9,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,21


## `OperationsProblemTemplate`s define `Stage`s
Sequential simulations in PowerSimulations are created by defining `OperationsProblems`
that represent `Stages`, and how information flows between executions of a `Stage` and
between different `Stage`s.

Let's start by defining a two stage simulation that might look like a typical day-Ahead
and real-time electricity market clearing process.

### We've already defined the reference model for the day-ahead unit commitment

In [3]:
#set_device_model!(template_ed, GenericBattery, BookKeeping)
template_uc


Operations Problem Specification
Transmission: CopperPlatePowerModel
Devices Models: 

	Type: ThermalStandard
 	Formulation: ThermalStandardUnitCommitment

	Type: PowerLoad
 	Formulation: StaticPowerLoad

	Type: HydroDispatch
 	Formulation: FixedOutput

	Type: RenewableFix
 	Formulation: FixedOutput

	Type: RenewableDispatch
 	Formulation: RenewableFullDispatch

	Type: HydroEnergyReservoir
 	Formulation: HydroDispatchRunOfRiver

Branches Models: 

	Type: Line
 	Formulation: StaticBranch

	Type: TapTransformer
 	Formulation: StaticBranch

	Type: Transformer2W
 	Formulation: StaticBranch

Services Models:

	Type: VariableReserve{ReserveDown}
 	Formulation: RangeReserve

	Type: VariableReserve{ReserveUp}
 	Formulation: RangeReserve



### Define the reference model for the real-time economic dispatch
In addition to the manual specification process demonstrated in the OperationsProblem
example, PSI also provides pre-specified templates for some standard problems:

In [4]:
template_ed = template_economic_dispatch()


Operations Problem Specification
Transmission: CopperPlatePowerModel
Devices Models: 

	Type: ThermalStandard
 	Formulation: ThermalDispatch

	Type: HydroDispatch
 	Formulation: HydroDispatchRunOfRiver

	Type: PowerLoad
 	Formulation: StaticPowerLoad

	Type: RenewableFix
 	Formulation: FixedOutput

	Type: RenewableDispatch
 	Formulation: RenewableFullDispatch

	Type: HydroEnergyReservoir
 	Formulation: HydroDispatchRunOfRiver

	Type: InterruptibleLoad
 	Formulation: InterruptiblePowerLoad

Branches Models: 

	Type: Line
 	Formulation: StaticBranch

	Type: TapTransformer
 	Formulation: StaticBranch

	Type: Transformer2W
 	Formulation: StaticBranch

	Type: HVDCLine
 	Formulation: HVDCDispatch

Services Models:

	Type: VariableReserve{ReserveDown}
 	Formulation: RangeReserve

	Type: VariableReserve{ReserveUp}
 	Formulation: RangeReserve



### Define the `SimulaitonProblems`
`OperationsProblem`s define models. The actual problem will change as the stage gets updated to represent
different time periods, but the formulations applied to the components is constant within
a stage. In this case, we want to define two stages with the `OperationsProblemTemplate`s
and the `System`s that we've already created.

In [5]:
problems = SimulationProblems(
        UC = OperationsProblem(template_uc, sys, optimizer = solver),
        ED = OperationsProblem(template_ed, sys_RT, optimizer = solver, balance_slack_variables = true),
    )

SimulationProblems(OrderedCollections.OrderedDict{Symbol,OperationsProblem}(:UC => OperationsProblem()
,:ED => OperationsProblem()
), [:UC, :ED])

Note that the "ED" problem has a `balance_slack_variables = true` argument. This adds slack
variables with a default penalty of 1e6 to the nodal energy balance constraint and helps
ensure feasibility with some performance impacts.

### `SimulationSequence`
Similar to an `OperationsProblemTemplate`, the `SimulationSequence` provides a template of
how to execute a sequential set of operations problems.

print_struct(SimulationSequence)

Let's review some of the `SimulationSequence` arguments.

### Chronologies
In PowerSimulations, chronologies define where information is flowing. There are two types
of chronologies.
 - inter-stage chronologies: Define how information flows between stages. e.g. day-ahead
solutions are used to inform economic dispatch problems
 - intra-stage chronologies: Define how information flows between multiple executions of a
single stage. e.g. the dispatch setpoints of the first period of an economic dispatch problem
are constrained by the ramping limits from setpoints in the final period of the previous problem.

Let's define an inter-stage chronology that synchronizes information from 24 periods of
the first stage with a set of executions of the second stage:

In [6]:
feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

Dict{Pair{String,String},Synchronize} with 1 entry:
  "UC"=>"ED" => Synchronize(24, 0, UpdateTrigger(-1, -1))

### `FeedForward` and `Cache`
The definition of exactly what information is passed using the defined chronologies is
accomplished with `FeedForward` and `Cache` objects. Specifically, `FeedForward` is used
to define what to do with information being passed with an inter-stage chronology. Let's
define a `FeedForward` that affects the semi-continuous range constraints of thermal generators
in the economic dispatch problems based on the value of the unit-commitment variables.

In [7]:
feedforward = Dict(
    ("ED", :devices, :ThermalStandard) => SemiContinuousFF(
        binary_source_problem = PSI.ON,
        affected_variables = [PSI.ACTIVE_POWER],
    ),
)

Dict{Tuple{String,Symbol,Symbol},SemiContinuousFF} with 1 entry:
  ("ED", :devices, :ThermalStandard) => SemiContinuousFF(:On, [:P], nothing)

### Sequencing
The stage problem length, look-ahead, and other details surrounding the temporal Sequencing
of stages are controlled using the `intervals` argument and the structure of the `Forecast`
data in the `System` of each problem.
 - intervals::Dict(String, Dates.Period) : defines the interval with which stage problems
advance after each execution. e.g. day-ahead problems have an interval of 24-hours

So, to define a typical day-ahead - real-time sequence, we can define the following:
 - Day ahead problems should represent 48 hours, advancing 24 hours after each execution (24-hour look-ahead)
 - Real time problems should represent 1 hour (12 5-minute periods), advancing 15 min after each execution (15 min look-ahead)

In [8]:
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Minute(15), Consecutive()))

Dict{String,Tuple{TimePeriod,Consecutive}} with 2 entries:
  "ED" => (Minute(15), Consecutive(UpdateTrigger(-1, -1)))
  "UC" => (Hour(24), Consecutive(UpdateTrigger(-1, -1)))

Finally, we can put it all together:

In [9]:
DA_RT_sequence = SimulationSequence(
    problems = problems,
    intervals = intervals,
    ini_cond_chronology = InterProblemChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
)

Feed Forward Chronology
-----------------------

ED: SemiContinuousFF -> ThermalStandard

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x96)   


## `Simulation`
Now, we can build and execute a simulation using the `SimulationSequence` and `Stage`s
that we've defined.

In [10]:
sim = Simulation(
    name = "rts-test",
    steps = 1,
    problems = problems,
    sequence = DA_RT_sequence,
    simulation_folder = dirname(dirname(pathof(SIIPExamples))),
)

Simulation()


### Build simulation

In [11]:
build!(sim)

┌ Info: 
│  ──────────────────────────────────────────────────────────────────────────────
│                                        Time                   Allocations      
│                                ──────────────────────   ───────────────────────
│        Tot / % measured:            4.80s / 100%            1.12GiB / 100%     
│ 
│  Section               ncalls     time   %tot     avg     alloc   %tot      avg
│  ──────────────────────────────────────────────────────────────────────────────
│  Build Simulation           1    4.80s   100%   4.80s   1.12GiB  100%   1.12GiB
│    Build Problems           1    4.63s  96.6%   4.63s   1.12GiB  100%   1.12GiB
│      Problem UC             1    3.41s  71.1%   3.41s    917MiB  79.6%   917MiB
│        ThermalStandard      1    794ms  16.5%   794ms    128MiB  11.1%   128MiB
│        RenewableDis...      1   92.8ms  1.93%  92.8ms   21.3MiB  1.85%  21.3MiB
│        Services             1   55.2ms  1.15%  55.2ms   22.4MiB  1.95%  22.4MiB
│   

BuildStatus.BUILT = 0

### Execute simulation
the following command returns the status of the simulation (0: is proper execution) and
stores the results in a set of HDF5 files on disk.

In [12]:
execute!(sim, enable_progress_bar = false)

Presolve 739 (-2561) rows, 1688 (-5872) columns and 3095 (-10933) elements
Perturbing problem by 0.001% of 10 - largest nonzero change 0.00010883922 ( 54.949531%) - largest zero change 0.00010866075
0  Obj 0.0074230668 Primal inf 364.26932 (451)
89  Obj 7341.4562 Primal inf 116.45055 (384)
178  Obj 11671.716 Primal inf 68.774479 (343)
267  Obj 13508.76 Primal inf 42.911762 (290)
356  Obj 15152.343 Primal inf 19.747797 (205)
445  Obj 16601.82 Primal inf 10.897706 (117)
534  Obj 17841.134 Primal inf 3.1276087 (37)
568  Obj 17841.135
Optimal - objective value 17841.11
After Postsolve, objective 17841.11, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 17841.10972 - 568 iterations time 0.012, Presolve 0.01
Presolve 739 (-2561) rows, 1688 (-5872) columns and 3095 (-10933) elements
Perturbing problem by 0.001% of 10 - largest nonzero change 0.00010883922 ( 54.949531%) - largest zero change 0.00010866075
0  Obj 0.0074230668 Primal inf 361.35223 (451)
89  Obj 7124.9675 Primal inf 

RunStatus.SUCCESSFUL = 0

## Results
To access the results, we need to load the simulation result metadata and then make
requests to the specific data of interest. This allows you to efficiently access the
results of interest without overloading resources.

In [13]:
results = SimulationResults(sim);
uc_results = get_problem_results(results, "UC"); # UC stage result metadata
ed_results = get_problem_results(results, "ED"); # ED stage result metadata

[ Info: checking integrity of /Users/cbarrows/Documents/repos/SIIPExamples.jl/rts-test/2/data_store/simulation_store.h5


Now we can read the specific results of interest for a specific problem, time window (optional),
and set of variables, duals, or parameters (optional)

In [14]:
read_variables(uc_results, names = [:P__ThermalStandard, :P__RenewableDispatch])

[ Info: reading variables from data store


Dict{Symbol,DataStructures.SortedDict{DateTime,DataFrame,Ord} where Ord<:Base.Order.Ordering} with 2 entries:
  :P__ThermalStandard   => DataStructures.SortedDict{DateTime,DataFrame,Base.Or…
  :P__RenewableDispatch => DataStructures.SortedDict{DateTime,DataFrame,Base.Or…

Or if we want the result of just one variable, parameter, or dual (must be defined in the
problem definition), we can use:

In [15]:
read_parameter(
    ed_results,
    :P__max_active_power__RenewableFix,
    initial_time = DateTime("2020-01-01T06:00:00"),
    count = 5,
)

[ Info: reading parameters from data store


DataStructures.SortedDict{DateTime,DataFrame,Base.Order.ForwardOrdering} with 5 entries:
  DateTime("2020-01-01T06:00:00") => 12×32 DataFrame…
  DateTime("2020-01-01T06:15:00") => 12×32 DataFrame…
  DateTime("2020-01-01T06:30:00") => 12×32 DataFrame…
  DateTime("2020-01-01T06:45:00") => 12×32 DataFrame…
  DateTime("2020-01-01T07:00:00") => 12×32 DataFrame…

* note that this returns the results of each execution step in a separate dataframe *
If you want the realized results (without lookahead periods), you can call `read_realized_*`:

In [16]:
read_realized_variables(uc_results, names = [:P__ThermalStandard, :P__RenewableDispatch])

[ Info: reading variables from data store


Dict{Symbol,DataFrame} with 2 entries:
  :P__ThermalStandard   => 24×65 DataFrame…
  :P__RenewableDispatch => 24×31 DataFrame…

## Plotting
Take a look at the [plotting examples.](../../notebook/3_PowerSimulations_examples/04_bar_stack_plots.ipynb)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*